In [1]:
from collections import defaultdict
import torch
import os   
import pandas as pd
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data, InMemoryDataset
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch.utils.data import random_split
from cap_dataset import CascadeRegression
from cap_model import GAT

In [2]:
print(torch.__version__)

2.4.0.dev20240417


In [3]:
print(os.getcwd())

/Users/rezatabrizi/dev/Repos/CAPP


In [4]:
fb_regression_ds = CascadeRegression(root="data", name="facebook", edge_index_path="data/raw/facebook/adj.txt", task="regression", observation=3)
fb_classification_ds = CascadeRegression(root="data", name="facebook", edge_index_path="data/raw/facebook/adj.txt", task="classification", observation=3)

In [5]:
sum(fb_classification_ds[0].y)

tensor(7)

In [6]:
model = GAT(fb_regression_ds.num_features, 64, 1, 8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

/Users/rezatabrizi/miniconda3/envs/ml/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:
total_size = len(fb_regression_ds)
train_size = int(0.7 * total_size)
valid_size = int(0.15 * total_size) 
test_size = total_size - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split (fb_regression_ds, [train_size, valid_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

/Users/rezatabrizi/miniconda3/envs/ml/lib/python3.8/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
(train_loader.dataset[2].y.shape)

torch.Size([1])

In [9]:
def mrse_loss(output, target):
    if target != 0:
        loss = ((output - target) / target) ** 2
    else:
        loss = (output - target) ** 2
    return loss

In [14]:
def train(dataset):
    total_loss = 0 
    model.train()
    for idx, data in enumerate(dataset):
        optimizer.zero_grad()
        out = model(data)
        current_MRSE = mrse_loss(out, data.y)
        #loss = F.mse_loss(out, data.y)
        print(data.y.shape)
        current_MRSE.backward()
        optimizer.step()
        total_loss += current_MRSE.item()
    return total_loss / len (dataset)

def test(dataset):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in dataset:
            out = model(data)
            current_MRSE = mrse_loss(out, data.y)
            total_loss += current_MRSE.item()
    return total_loss / len(dataset)

In [15]:
for epoch in range(100):
    train_loss = train(train_dataset)
    valid_loss = test(valid_dataset)
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

test_loss = test(test_dataset)
print(f'Test MSE Loss: {test_loss:.4f}')

torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])
torch.Size([4039, 1])
torch.Size([1])
torch.Size([1])


KeyboardInterrupt: 

1. Reduce head to have better performance
2. Run 100 epochs 
3. Use graph norm 
4. See how coupledgnn data is 
5. see if there is an issue with the y and the output

Lab meetin 5/04 
1. Remove graph pooling and just sum the nx1 matrix after decoding. After decoding apply sigmoid. 
2. Download weibo dataset and preprocess
3. Remove activation between layers. Only needed at the end. 
4. make sure code is right because it is only possible to get fluctutation if your code is not right. 